In [2]:
import pandas as pd
df = pd.read_csv('AWCustomers.csv')
df.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18361 entries, 0 to 18360
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   CustomerID            18361 non-null  int64 
 1   Title                 101 non-null    object
 2   FirstName             18361 non-null  object
 3   MiddleName            10572 non-null  object
 4   LastName              18361 non-null  object
 5   Suffix                3 non-null      object
 6   AddressLine1          18361 non-null  object
 7   AddressLine2          311 non-null    object
 8   City                  18361 non-null  object
 9   StateProvinceName     18361 non-null  object
 10  CountryRegionName     18361 non-null  object
 11  PostalCode            18361 non-null  object
 12  PhoneNumber           18361 non-null  object
 13  BirthDate             18361 non-null  object
 14  Education             18361 non-null  object
 15  Occupation            18361 non-null

In [9]:
df_selected=df[['City','StateProvinceName','CountryRegionName','BirthDate','Education','Occupation','Gender','MaritalStatus','HomeOwnerFlag','NumberCarsOwned','NumberChildrenAtHome','TotalChildren','YearlyIncome']].copy()
df_selected['BirthDate'] = pd.to_datetime(df_selected['BirthDate'])
df_selected['Age'] = (pd.Timestamp.today() - df_selected['BirthDate']).dt.days // 365
df_selected = df_selected.drop(columns=['BirthDate'])
df_selected.info()
df_selected.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18361 entries, 0 to 18360
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   City                  18361 non-null  object
 1   StateProvinceName     18361 non-null  object
 2   CountryRegionName     18361 non-null  object
 3   Education             18361 non-null  object
 4   Occupation            18361 non-null  object
 5   Gender                18361 non-null  object
 6   MaritalStatus         18361 non-null  object
 7   HomeOwnerFlag         18361 non-null  int64 
 8   NumberCarsOwned       18361 non-null  int64 
 9   NumberChildrenAtHome  18361 non-null  int64 
 10  TotalChildren         18361 non-null  int64 
 11  YearlyIncome          18361 non-null  int64 
 12  Age                   18361 non-null  int64 
dtypes: int64(6), object(7)
memory usage: 1.8+ MB


,City,StateProvinceName,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Age
0,Wollongong,New South Wales,Australia,Bachelors,Clerical,M,M,1,3,0,1,81916,37
1,Shawnee,British Columbia,Canada,Partial College,Clerical,M,M,1,2,1,2,81076,53
2,West Covina,California,United States,Bachelors,Clerical,F,S,0,3,0,0,86387,39
3,Liverpool,England,United Kingdom,Partial College,Skilled Manual,M,M,1,2,1,2,61481,47
4,Werne,Nordrhein-Westfalen,Germany,Partial College,Skilled Manual,M,S,1,1,0,0,51804,50


In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler

In [11]:
for col in ['City','StateProvinceName','CountryRegionName','Education','Occupation','Gender','MaritalStatus']:
    df_selected[col] = df_selected[col].fillna('Unknown')

for col in ['Age','HomeOwnerFlag','NumberCarsOwned','NumberChildrenAtHome','TotalChildren','YearlyIncome']:
    df_selected[col] = df_selected[col].fillna(df_selected[col].median())


minmax = MinMaxScaler()
df_selected[['Age','YearlyIncome']] = minmax.fit_transform(df_selected[['Age','YearlyIncome']])

std=StandardScaler()
df_selected[['NumberCarsOwned','NumberChildrenAtHome','TotalChildren']] = std.fit_transform(
    df_selected[['NumberCarsOwned','NumberChildrenAtHome','TotalChildren']]
)

bin=Binarizer(threshold=0)
df_selected[['HomeOwnerFlag']] = bin.fit_transform(df_selected[['HomeOwnerFlag']])

df_selected = pd.get_dummies(df_selected, columns=['City', 'StateProvinceName', 'CountryRegionName','Education', 'Occupation', 'Gender', 'MaritalStatus'])


In [13]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.stats import pearsonr

In [ ]:
obj1 = df_selected.iloc[0].values.reshape(1, -1)
obj2 = df_selected.iloc[1].values.reshape(1, -1)

binary_cols = [col for col in df_selected.columns if set(df_selected[col].unique()).issubset({0, 1})]
binary_data = df_selected[binary_cols]

obj1_bin = binary_data.iloc[0].values
obj2_bin = binary_data.iloc[1].values
simple_matching = np.sum(obj1_bin == obj2_bin) / len(obj1_bin)

intersection = np.sum(np.logical_and(obj1_bin, obj2_bin))
union = np.sum(np.logical_or(obj1_bin, obj2_bin))
jaccard = intersection / union if union != 0 else 0

cos_sim = cosine_similarity(obj1, obj2)[0][0]

print(f"Simple Matching: {simple_matching:.4f}")
print(f"Jaccard: {jaccard:.4f}")
print(f"Cosine: {cos_sim:.4f}")

Simple Matching: 0.9767
Jaccard: 0.3333
Cosine: 0.4416
